# download data:
i was unable to find ftp login info, so this has to be done by hand ( if you find a way around it lmk) 

i also could not get the noaa python package to work - https://pypi.org/project/noaa-weather/

the dataset we are using comes from here: https://www.ncei.noaa.gov/metadata/geoportal/rest/metadata/item/gov.noaa.ncdc:C00861/html

1. you can read on the features of the library and the important files here: ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt
2. the main directory you will be using is ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/
3. recommend that you download ghcnd-stations.txt, ghcnd-states.txt, and ghcnd_ghc.tar.gz  (previously ghcnd_all.tar.gz  but that is 10 times larger)

In [ ]:
import pandas as pd

In [21]:
# gather weather station metadata from the stations.txt file

f = open("F:\weather\ghcnd-stations.txt","r")
lines = f.readlines()

i = 0
colnames = ['ID', 'LAT', 'LON', 'ELEV', 'STATE', 'NAME', 'GSN', 'HCNCRN', 'WMOID']
stationlist = []
stations = pd.DataFrame(columns=colnames)
for line in lines:
    # first 2 characters are the country code , we only care about us stations
    if line[0:2] == 'US':
        
        # the description of the file seemed slightly off, i tested and found these column numbers to work best
        row = {"ID": line[0:11],
                "LAT": float(line[13:20]),
                "LON": float(line[21:30]),
                "ELEV": float(line[31:37]),
                "STATE": line[38:40],
                "NAME": line[41:71],
                "GSN": line[72:75],
                "HCNCRN": line[76:79],
                "WMOID": line[80:85]
               }
        stationlist.append(row)
    else:
        pass
stations = stations.append(stationlist)
f.close()
stations

,ID,LAT,LON,ELEV,STATE,NAME,GSN,HCNCRN,WMOID
0,US009052008,43.7333,-96.6333,482.0,SD,SIOUX FALLS (ENVIRON. CANADA),,,
1,US10RMHS145,40.5268,-105.1113,1569.1,CO,RMHS 1.6 SSW,,,
2,US10adam001,40.5680,-98.5069,598.0,NE,JUNIATA 1.5 S,,,
3,US10adam002,40.5093,-98.5493,601.1,NE,JUNIATA 6.0 SSW,,,
4,US10adam003,40.4663,-98.6537,615.1,NE,HOLSTEIN 0.1 NW,,,
...,...,...,...,...,...,...,...,...,...
66384,USW00096405,60.4731,-145.3542,25.3,AK,CORDOVA 14 ESE,,,
66385,USW00096406,64.5014,-154.1297,78.9,AK,RUBY 44 ESE,,,70224
66386,USW00096407,66.5620,-159.0036,6.7,AK,SELAWIK 28 E,,,70170
66387,USW00096408,63.4519,-150.8747,678.2,AK,DENALI 27 N,,,70244


fstates = open("F:\weather\ghcnd-states.txt","r")
lines3 = fstates.readlines()

colnames = ['CODE', 'NAME']
states = pd.DataFrame(columns=colnames)
for line in lines3:
    modline = line.strip('\n')
    data = {'CODE': line[0:2],
            "NAME": modline[3:50]
           }
    states = states.append(data, ignore_index=True)    
states

In [23]:
fstates = open("F:\weather\ghcnd-states.txt","r")
lines3 = fstates.readlines()

colnames = ['CODE', 'NAME']
states = pd.DataFrame(columns=colnames)
for line in lines3:
    modline = line.strip('\n')
    data = {'CODE': line[0:2],
            "NAME": modline[3:50]
           }
    states = states.append(data, ignore_index=True)    
states


,CODE,NAME
0,AB,ALBERTA
1,AK,ALASKA
2,AL,ALABAMA
3,AR,ARKANSAS
4,AS,AMERICAN SAMOA
...,...,...
69,WA,WASHINGTON
70,WI,WISCONSIN
71,WV,WEST VIRGINIA
72,WY,WYOMING


In [24]:
stationplus = stations.join(states.set_index('CODE'), on='STATE', rsuffix='_STATE')
stationplus

,ID,LAT,LON,ELEV,STATE,NAME,GSN,HCNCRN,WMOID,NAME_STATE
0,US009052008,43.7333,-96.6333,482.0,SD,SIOUX FALLS (ENVIRON. CANADA),,,,SOUTH DAKOTA
1,US10RMHS145,40.5268,-105.1113,1569.1,CO,RMHS 1.6 SSW,,,,COLORADO
2,US10adam001,40.5680,-98.5069,598.0,NE,JUNIATA 1.5 S,,,,NEBRASKA
3,US10adam002,40.5093,-98.5493,601.1,NE,JUNIATA 6.0 SSW,,,,NEBRASKA
4,US10adam003,40.4663,-98.6537,615.1,NE,HOLSTEIN 0.1 NW,,,,NEBRASKA
...,...,...,...,...,...,...,...,...,...,...
66384,USW00096405,60.4731,-145.3542,25.3,AK,CORDOVA 14 ESE,,,,ALASKA
66385,USW00096406,64.5014,-154.1297,78.9,AK,RUBY 44 ESE,,,70224,ALASKA
66386,USW00096407,66.5620,-159.0036,6.7,AK,SELAWIK 28 E,,,70170,ALASKA
66387,USW00096408,63.4519,-150.8747,678.2,AK,DENALI 27 N,,,70244,ALASKA
